<a href="https://colab.research.google.com/github/darja/NeuralUni/blob/master/20_genetic_mnist_ryazhskikh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import mnist
import numpy as np
from numpy.random import randint
from random import sample
from keras.utils import to_categorical
import gc

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Reshape, MaxPooling2D, Dropout, Flatten

Using TensorFlow backend.


In [34]:
a = [[1, 11, 111], [2, 22, 222]]
b = [[1111, 11111], [2222, 22222]]
np.append(a, b, axis=1)

array([[    1,    11,   111,  1111, 11111],
       [    2,    22,   222,  2222, 22222]])

Загрузка данных

In [0]:
# В Keras встроены средства работы с популярными наборами данных
(x_train_org, y_train_org), (x_test_org, y_test_org) = mnist.load_data()

x_train = x_train_org.reshape(60000, 784)
# x_test = x_test_org.reshape(10000, 784)

# Векторизованные операции
# Применяются к каждому элементу массива отдельно
x_train = x_train.astype('float32')
x_train = x_train / 255 

# Преобразуем метки в формат one hot encoding
y_train = to_categorical(y_train_org, 10)

In [0]:
x_train = x_train[:100]
y_train = y_train[:100]

# del x_test_org
# del y_test_org
del x_train_org
del y_train_org

In [0]:
del x_test

Бот и сопутствующее

In [0]:
bot_functions = [
    lambda vec: np.mean(vec), # 0 – среднее
    lambda vec: np.max(vec)   # 1 – минимум
]

img_size = 28 * 28
target_size = (42, 42, 1)
features_count = target_size[0] * target_size[1] - img_size # число добавляемых к основному датасету признаков
feature_size = 10 # число элементов, участвующих в формировании признака
bot_item_size = feature_size + 1 # размер компонента бота (индексы элементов + операция)
bot_size = features_count * bot_item_size # итоговый размер массива-представления бота

model = Sequential()
x_train_extended = []

def train_neural_network(x_train, y_train):
    model = Sequential()

    model.add(Reshape(target_size, input_shape=(img_size + features_count,)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(2000, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    history = model.fit(x_train, y_train, epochs=3, batch_size=200, validation_split=0.05, verbose=0)
    acc = history.history["val_acc"][-1]
    print(acc)
    return acc

class Bot:
    def __init__(self, parents=[], mutation_rate=0):
        self.score = 0

        if len(parents) == 0:
            # Создание случайного бота
            self.cells = np.zeros(bot_size, dtype="uint8")
            for i in range(features_count):
                self.add_feature(i)
        else:
            # Создание бота-потомка
            produce_cells = np.vectorize(lambda i: self.inherit_cell(parents, mutation_rate, i))

            self.cells = produce_cells(np.arange(bot_size))

    def add_feature(self, conv_index):
        conv_start = conv_index * bot_item_size
        self.cells[conv_start] = randint(0, len(bot_functions))  # операция над элементами
        conv_content = sample(range(img_size), feature_size)
        for i in range(feature_size):
            self.cells[conv_start + i + 1] = int(conv_content[i])  # индексы элементов

    def __str__(self):
        return f"{self.score} | {self.cells}"

    # наследование одной ячейки
    def inherit_cell(self, parents, mutation_rate, cell_index):
        mutate = np.random.rand()
        if mutate > mutation_rate:
            rnd = randint(len(parents))
            return parents[rnd].cells[cell_index]
        else:
            if cell_index % bot_item_size == 0:
                return randint(0, len(bot_functions))  # операция над элементами
            else:
                return randint(img_size)

    def extend_image(self, img):
        extension = []
        for i in range(features_count):
            func_index = self.cells[i * bot_item_size]
            if func_index > 1:
                print(self.cells.shape)
            func = bot_functions[func_index]
            vec = img[self.cells[i * bot_item_size + 1: (i + 1) * bot_item_size]]
            value = func(vec)
            extension.append(value)
        return extension

    def calculate_score(self):
        x_train_extended = []

        # запуск нейросети
        for i in range(len(x_train)):
            ext = self.extend_image(x_train[i])
            x_train_extended_i = np.append(x_train[i], ext)
            x_train_extended.append(x_train_extended_i)

        x_train_extended = np.array(x_train_extended)
        self.score = train_neural_network(x_train_extended, y_train)
        del x_train_extended


Эволюция

In [15]:
import gc
import resource

def mem():
    print('Memory usage         : % 2.2f MB' % round(
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0,1)
    )

mem()
del model

print('GC collected objects : %d' % gc.collect())

mem()

Memory usage         :  2.90 MB


NameError: ignored

In [0]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [26]:
get_size(x_train_extended)

64

In [21]:
whos

Variable               Type                          Data/Info
--------------------------------------------------------------
Bot                    type                          <class '__main__.Bot'>
Conv2D                 type                          <class 'tensorflow.python<...>rs.convolutional.Conv2D'>
Dense                  type                          <class 'tensorflow.python<...>keras.layers.core.Dense'>
Dropout                type                          <class 'tensorflow.python<...>ras.layers.core.Dropout'>
Evolution              type                          <class '__main__.Evolution'>
Flatten                type                          <class 'tensorflow.python<...>ras.layers.core.Flatten'>
MaxPooling2D           type                          <class 'tensorflow.python<...>rs.pooling.MaxPooling2D'>
Reshape                type                          <class 'tensorflow.python<...>ras.layers.core.Reshape'>
Sequential             type                          <class 't

In [0]:
class Evolution:
    calculate_score = np.vectorize(lambda bot: bot.calculate_score())
    get_scores = np.vectorize(lambda bot: bot.score)

    def __init__(self, size=100, survivors=50, mutation=0.5, parents_count=2):
        # размер популяции
        self.size = size

        # сколько выживает на каждом шаге
        self.survivors = survivors

        # сколько создаётся на каждом шаге
        self.new = self.size - self.survivors

        # доля мутации
        self.mutation = mutation

        # от скольки родителей будут наследоваться новые боты
        self.parents_count = parents_count

        # популяция ботов: случайное распределение
        self.generation = np.vectorize(lambda i: Bot())(np.arange(0, self.size))
        Evolution.calculate_score(self.generation)
        

    def stat(self):
        total_score = np.vectorize(lambda bot: bot.score)(self.generation)
        mean_val = sum(total_score) / self.size
        max_val = np.max(total_score)

        return mean_val, max_val

    def sorted_scores(self):
        scores = Evolution.get_scores(self.generation)
        return sorted(scores)

    def next_generation(self):
        sorted_bots = sorted(self.generation, key=lambda b: -b.score)
        new_gen = np.array(sorted_bots[:self.survivors])
#         np.random.shuffle(new_gen)
        for i in range(self.new):
            # родителями могут быть только выжившие на текущей итерации
            parent_ids = sample(range(self.survivors), self.parents_count)
            parents = new_gen[parent_ids]
            bot = Bot(parents, self.mutation)
            new_gen = np.append(new_gen, bot)

        self.generation = new_gen
        

    def best_bot(self):
        return sorted(self.generation, key=lambda b: -b.score)[0]

evo = Evolution(size=10, survivors=5, mutation=0.1, parents_count=2)

for epoch in range(10):
    print(f"Epoch {epoch}")
    epoch_mean, epoch_max = evo.stat()
    print(f"{epoch}\t Mean {epoch_mean}, max {epoch_max}\t{evo.sorted_scores()}")
    evo.next_generation()
#     for bot in evo.generation:
#         print(bot.cells)
    Evolution.calculate_score(evo.generation)
    gc.collect()

In [105]:
for bot in evo.generation:
    print(bot.score)
#     print(bot.cells[np.array(range(features_count)) * bot_item_size])
#     print(bot.cells[10])


0
0
0
0
0


In [0]:
evo.bot

In [113]:
bt1 = Bot()
bt2 = Bot()

print(bt1.cells.shape)
print(bt2.cells.shape)
bt3 = Bot([bt1, bt2])
print(bt3.cells.shape)

(10780,)
(10780,)
(10780,)


In [95]:
bt3 = Bot([bt3, bt2], 0.1)
print(bt3.cells[np.array(range(features_count)) * bot_item_size])

[  1   1   1   0   0   0   0   0 184   1   0   0   0   1   0   1   1 200
   0   0   1   0   1 203   1   1   1   1   1   1   1   1   1   0   1   1
   1   1   0 556   0   1   0 342 393   1   0   1 313   0   1 680   1   0
   1   1   0   1   0   0   0   1   0   1   0 138   0  82   1   0   1   0
   1   1   0   0   1   0   0   0   1  48   1   0   1   0   0   0   1   0
   1   0   0   1   0   1   1   1   1   1   0   1   0   1   1   0   0   1
   1   1   0   0   1   0   0   1   1   0   0 419   1 541   0   1   0   1
 519   1   1   0   0   0   1  14   1   0 544   0   0   1   0   0  37   1
   1  60   1   1   0   0   1   0   1   0   0   1   0   1   0   0   0   1
   0   0   0   0   1   1   0   0   0 194   0   1   1   0   1 260   0   0
   1   1   1   0   0   0   1   0   0   0   1   0   0   0   0 207   1   1
   0   1   1   1   0   0   0 206   1   0   1 625   1   1   0   0 416   1
   1   1 181   1   0   0   0   0  94   1   1   1   0   1   1   0 762   0
   0   0   1   0   0   0   1   1 450   0   1   1   